# optimus-challenge

O arquivo totalizador.csv contém amostras do valor totalizado, realizadas no período de 5 dias.

Usando Pandas, é necessário descobrir:

1. quantas toneladas foram produzidas nesse período;

1. nesse período, por quantas horas a produção permaneceu parada, sabendo-se que, com produção em andamento, a balança sempre registra pelo menos 3 toneladas a cada 5 minutos.

In [1]:
import pandas as pd
import numpy as np
from datetime import date

In [2]:
totalizador_df = pd.read_csv('./totalizador.csv')

print('Formato do dataframe:', totalizador_df.shape, '\n')
totalizador_df.head(5)

Formato do dataframe: (28584, 1) 



,t;Totalizador
0,2018-03-01 00:00:07.505000-03:00;329
1,2018-03-01 00:00:22.512000-03:00;330
2,2018-03-01 00:00:37.504000-03:00;330
3,2018-03-01 00:00:52.496000-03:00;331
4,2018-03-01 00:01:07.503000-03:00;331


In [3]:
totalizador_df.describe()

,t;Totalizador
count,28584
unique,28584
top,2018-03-03 12:32:22.510000-03:00;2018
freq,1


In [4]:
def get_date(line):
    ano, mes, dia = (str(line).split()[0].split('-'))
    return date(int(ano), int(mes), int(dia))

def get_time(line):
    return str(line).split()[1].split('-')[0].split('.')[0]

def get_tons(line):
    return int(str(line).split(';')[-1])

In [5]:
totalizador_df['data'] = totalizador_df['t;Totalizador'].apply(get_date)
totalizador_df['hora'] = totalizador_df['t;Totalizador'].apply(get_time)
totalizador_df['toneladas'] = totalizador_df['t;Totalizador'].apply(get_tons)

tons_df = totalizador_df[['data', 'hora', 'toneladas']]

In [6]:
tons_df.head()

,data,hora,toneladas
0,2018-03-01,00:00:07,329
1,2018-03-01,00:00:22,330
2,2018-03-01,00:00:37,330
3,2018-03-01,00:00:52,331
4,2018-03-01,00:01:07,331


In [7]:
# Conferindo as datas
print('Quantidade de dias:', tons_df['data'].nunique())
print('Data mínima:', tons_df['data'].min())
print('Data máxima:', tons_df['data'].max())

Quantidade de dias: 5
Data mínima: 2018-03-01
Data máxima: 2018-03-05


In [8]:
CINCO = 5 * 60

def time_to_sec(line):
    hora, minuto, segundo = line.split(':')
    return ((int(hora) * 3600) + (int(minuto) * 60) + int(segundo))

def delta_time(dh1, dh2):
    dt1 = (dh1[0].day * 86400) + dh1[1]
    dt2 = (dh2[0].day * 86400) + dh2[1]
    return dt2 - dt1

In [9]:
tons_df['timesec'] = tons_df['hora'].apply(time_to_sec)
tons_df.head()

,data,hora,toneladas,timesec
0,2018-03-01,00:00:07,329,7
1,2018-03-01,00:00:22,330,22
2,2018-03-01,00:00:37,330,37
3,2018-03-01,00:00:52,331,52
4,2018-03-01,00:01:07,331,67


In [11]:
tons_df['dtime'] = 0

for n in range(1, tons_df.shape[0]):
    delta = delta_time([tons_df.loc[n-1,'data'], tons_df.loc[n-1,'timesec']], \
                       [tons_df.loc[n,'data'], tons_df.loc[n,'timesec']])
    if delta > 0:
        tons_df.loc[n,'dtime'] = delta
    
tons_df.head()

,data,hora,toneladas,timesec,dtime
0,2018-03-01,00:00:07,329,7,0
1,2018-03-01,00:00:22,330,22,15
2,2018-03-01,00:00:37,330,37,15
3,2018-03-01,00:00:52,331,52,15
4,2018-03-01,00:01:07,331,67,15


In [12]:
tons_df['dtime'].unique()

array([  0,  15, 150, 135, 300, 120, 240, 105, 195, 255, 225, 315, 165])

In [13]:
stand_by = tons_df[tons_df['dtime']>=CINCO]
stand_by

,data,hora,toneladas,timesec,dtime
2435,2018-03-01,10:17:52,1461,37072,300
9751,2018-03-02,17:11:07,1865,61867,315
10171,2018-03-02,19:00:52,2103,68452,300


In [10]:
toneladas = 0

for n in range(0, tons_df.shape[0]-1):
    if tons_df.loc[n,'toneladas'] > tons_df.loc[n+1,'toneladas']:
        toneladas = toneladas + tons_df.loc[n,'toneladas']
        
print('Toneladas produzidas entre', tons_df['data'].min(), 'e', tons_df['data'].max(), ':', toneladas)

Toneladas produzidas entre 2018-03-01 e 2018-03-05 : 13839


In [27]:
tempo_parado = 0
init = 0

while init < tons_df.shape[0]-1:
    for n in range(init, tons_df.shape[0]):
        for m in range(n+1, tons_df.shape[0]):
            tempo = tons_df.loc[m,'timesec'] - tons_df.loc[n,'timesec']
            if tempo >= CINCO:
                init = m+1
                delta_tons = tons_df.loc[m,'toneladas'] - tons_df.loc[n,'toneladas']
                if delta_tons < 3:
                    tempo_parado = tempo_parado + tempo
                    break
        break

    
horas = tempo_parado // 3600
minutos = (tempo_parado - (horas * 3600)) // 60
segundos = tempo_parado % 60

print('Total de horas paradas:', '{}:{:02}:{:02}'.format(horas, minutos, segundos))

Total de horas paradas: 21:04:15
